# NLP - Trabajo Práctico 1
Clasificador de Recomendaciones Recreativas utilizando Procesamiento de Lenguaje Natural (NLP)

Alumno: Pistelli, Pablo

2024





## Enunciado

### Contexto

Una persona dentro de un mes, se tomará 15 días de vacaciones en la playa. Sin embargo, se estima que durante al menos cuatro de esos días habrá lluvias, lo que podría limitar las actividades al aire libre. Para esos días de mal clima, se propone una solución que facilite la recreación en función del estado de ánimo del día.

### Objetivo

Desarrollar un programa de Procesamiento de Lenguaje Natural que, según el estado de ánimo del usuario, recomiende entre ver una película, jugar un juego de mesa o leer un libro (o varias opciones para cada caso). Para ello, deberá construir un clasificador que categorice el estado de ánimo del usuario. Luego sugerir el conjunto de recomendaciones basada en una frase de preferencia ingresada por el usuario.

### Pasos para la construcción del proyecto
**1. Clasificación del Estado de Ánimo:**

  Utilice los conocimientos aprendidos en la Unidad 3 para desarrollar un clasificador a partir de un prompt con el que determine el estado de ánimo del usuario, el cual deberá categorizarse por ejemplo: "Alegre", "Melancólico" o "Ni fu ni fa".

**2. Ingreso de Preferencias:**

  Una vez determinado el estado de ánimo, el usuario deberá ingresar una frase que describa la temática que le gustaría explorar. Por ejemplo: "una historia de amor en la selva".

**3. Búsqueda de Opciones:**

  El programa deberá comparar la frase ingresada por el usuario con diversas estructuras de texto provenientes de diferentes fuentes de datos utilizando los métodos aprendidos en clase.

  Disponga de los siguientes datasets:

  * bgg_database.csv: Base de datos de juegos de mesa.
  * IMDB-Movie-Data.csv: Base de datos de películas.
  * Libros del Proyecto Gutenberg: Realice web scraping para conformar un dataset con información sobre los 1000 libros más populares del Proyecto Gutenberg. El enlace a utilizar es el siguiente: https://www.gutenberg.org/browse/scores/top1000.php#books-last1.

**4. Recomendaciones:**

  Con base en el estado de ánimo del usuario y la frase ingresada, el programa deberá ofrecer recomendaciones pertinentes entre películas, juegos de mesa o libros. Utilice las herramientas de NLP aprendidas en las tres primeras unidades para lograr resultados coherentes y personalizados.



### Requerimientos mínimos

* Utilice clasificadores para determinar el estado de ánimo (por ejemplo, métodos de clasificación supervisada).

* Aplique técnicas de embeddings y comparación de similitud semántica para encontrar las mejores coincidencias en los datasets.

* Utilice la potencia de reconocimiento de entidades nombradas, (NER, modelo Gliner) con el objetivo de obtener los mejores resultados buscados.


### Entrega

* Se debe entregar un informe donde se documente cómo se implementa cada parte del programa, incluyendo explicaciones de cómo funcionan los algoritmos utilizados.

* Realice pruebas con diferentes ejemplos para mostrar la efectividad del clasificador y del sistema de recomendación.

* El código debe estar bien comentado y seguir buenas prácticas de programación. Debe utilizar entregar el o los colab utilizados en formato de notebook.

* Se debe entregar el dataset generado con el web scraping.


**Nota:** Las fuentes de datos se encuentran en inglés, la aplicación debe comunicarse con el usuario en español.

**Opcional:** Puede presentar una aplicación para el programa desarrollado, utilizando Google Colab con una interfaz sencilla basada en widgets como los proporcionados por la librería *ipywidgets*.


## Desarrollo

### Requerimientos

In [1]:
# Clasificadores de estado de ánimo
# Modelo 1
!pip install transformers
# Modelo 2
!pip install pysentimiento

# Idiomas
!pip install deep-translator

# Búsqueda en bases de datos
!pip install sentence-transformers

# Reconocimiento de entidades nombradas en bases de datos
!pip install gliner

# Interacción con el usuario
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.5 MB/s eta 0:00:00
 

In [2]:
# Clasificadores de estado de ánimo
  # Modelo 1
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
  # Modelo 2
from pysentimiento import create_analyzer

# Idiomas
from deep_translator import GoogleTranslator
from transformers import MarianMTModel, MarianTokenizer

# Bases de datos
import pandas as pd

# Búsqueda en bases de datos
from sentence_transformers import SentenceTransformer, util
import torch

# Manejo de embeddings
import numpy as np

# Reconocimiento de entidades nombradas en bases de datos
from gliner import GLiNER

# Interfaz de usuario
import ipywidgets as widgets
from IPython.display import display, clear_output

### Directorio Data

In [3]:
# Setear ruta a directorio Data
data_dir = '/content/drive/MyDrive/TUIA/Procesamiento lenguaje Natural/TP1/Data/'

juegos_dir = data_dir + 'bgg_database_NER.csv'
peliculas_dir = data_dir + 'IMDB-Movie-Data_NER.csv'
libros_dir = data_dir + 'Top1000_EBooks_ProjectGutemberg_NER.csv'
embeddings_juegos_dir = data_dir + 'incrustaciones_juegos.npy'
embeddings_peliculas_dir = data_dir + 'incrustaciones_peliculas.npy'
embeddings_libros_dir = data_dir + 'incrustaciones_libros.npy'

### Bases de datos procesadas

In [4]:
# Bases de datos
juegos = pd.read_csv(juegos_dir)
peliculas = pd.read_csv(peliculas_dir)
libros = pd.read_csv(libros_dir)

#### Juegos

In [5]:
juegos.head()

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories,entities
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.415,8.60,46836.0,Brass: Birmingham is an economic strategy game...,2018,2,4,60,120,14,3.8776,"[{'min': 3, 'max': 4}]","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']","['Hand Management', 'Income', 'Loans', 'Market...","['Age of Reason', 'Economic', 'Industry / Manu...","[{'start': 57, 'end': 71, 'text': 'Martin Wall..."
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.377,8.53,53807.0,Pandemic Legacy is a co-operative campaign gam...,2015,2,4,60,60,13,2.8308,"[{'min': 4, 'max': 4}]","['Rob Daviau', 'Matt Leacock']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Environmental', 'Medical']","[{'start': 0, 'end': 15, 'text': 'Pandemic Leg..."
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.349,8.59,62592.0,Gloomhaven is a game of Euro-inspired tactica...,2017,1,4,60,120,14,3.9132,"[{'min': 3, 'max': 3}]",['Isaac Childres'],"['Action Queue', 'Action Retrieval', 'Campaign...","['Adventure', 'Exploration', 'Fantasy', 'Fight...","[{'start': 25, 'end': 54, 'text': 'Euro-inspir..."
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.335,8.54,44728.0,"In Ark Nova, you will plan and design a modern...",2021,1,4,90,150,14,3.7653,"[{'min': 2, 'max': 2}]",['Mathias Wigge'],"['Action Queue', 'End Game Bonuses', 'Grid Cov...","['Animals', 'Economic', 'Environmental']","[{'start': 3, 'end': 11, 'text': 'Ark Nova', '..."
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.240,8.60,24148.0,Twilight Imperium (Fourth Edition) is a game o...,2017,3,6,240,480,14,4.3173,"[{'min': 6, 'max': 6}]","['Dane Beltrami', 'Corey Konieczka', 'Christia...","['Action Drafting', 'Area-Impulse', 'Dice Roll...","['Civilization', 'Economic', 'Exploration', 'N...","[{'start': 331, 'end': 347, 'text': 'Ghosts of..."


In [6]:
# Extraigo categorías posibles de juegos para hacer una primera selección según estado de ánimo
juegos_categories_list=[]

for categories in juegos['categories']:
  categories_split = []
  categories_split = categories.replace('[', '').replace(']', '').replace("'", '').split(', ')

  for category in categories_split:
    if category not in juegos_categories_list:
      juegos_categories_list.append(category)

In [7]:
len(juegos_categories_list)

81

In [8]:
juegos_categories_list

['Age of Reason',
 'Economic',
 'Industry / Manufacturing',
 'Post-Napoleonic',
 'Trains',
 'Transportation',
 'Environmental',
 'Medical',
 'Adventure',
 'Exploration',
 'Fantasy',
 'Fighting',
 'Miniatures',
 'Animals',
 'Civilization',
 'Negotiation',
 'Political',
 'Science Fiction',
 'Movies / TV / Radio theme',
 'Novel-based',
 'Space Exploration',
 'Territory Building',
 'Wargame',
 'Civil War',
 'Mythology',
 'Modern Warfare',
 'Card Game',
 'American West',
 'Dice',
 'Medieval',
 'Ancient',
 'City Building',
 'Horror',
 'Farming',
 'Puzzle',
 'Nautical',
 'Collectible Components',
 'Travel',
 'Educational',
 'Religious',
 'Deduction',
 'Racing',
 'Sports',
 'Comic Book / Strip',
 'Spies/Secret Agents',
 'Action / Dexterity',
 'Murder/Mystery',
 'Pirates',
 'Bluffing',
 'Video Game Theme',
 'Mature / Adult',
 'Abstract Strategy',
 'Renaissance',
 'Arabian',
 'Aviation / Flight',
 'Prehistoric',
 'Party Game',
 'Word Game',
 'World War II',
 'Number',
 'Pike and Shot',
 'Real-ti

#### Películas

In [9]:
peliculas.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,entities
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76,"[{'start': 11, 'end': 34, 'text': 'intergalact..."
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65,"[{'start': 80, 'end': 84, 'text': 'moon', 'lab..."
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62,"[{'start': 0, 'end': 11, 'text': 'Three girls'..."
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59,"[{'start': 5, 'end': 29, 'text': 'city of huma..."
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40,"[{'start': 0, 'end': 26, 'text': 'A secret gov..."


In [10]:
# Extraigo géneros posibles de películas para hacer una primera selección según estado de ánimo
peliculas_genres_list=[]

for genres in peliculas['Genre']:
  genres_split = []
  genres_split = genres.replace('[', '').replace(']', '').replace("'", '').split(',')

  for genre in genres_split:
    if genre not in peliculas_genres_list:
      peliculas_genres_list.append(genre)

In [11]:
len(peliculas_genres_list)

20

In [12]:
peliculas_genres_list

['Action',
 'Adventure',
 'Sci-Fi',
 'Mystery',
 'Horror',
 'Thriller',
 'Animation',
 'Comedy',
 'Family',
 'Fantasy',
 'Drama',
 'Music',
 'Biography',
 'Romance',
 'History',
 'Crime',
 'Western',
 'War',
 'Musical',
 'Sport']

#### Libros

In [13]:
libros.head()

,Title,Author,Summary,Subjects,URL,entities
0,"Frankenstein; Or, The Modern Prometheus","Shelley, Mary Wollstonecraft, 1797-1851","""Frankenstein; Or, The Modern Prometheus"" by M...","['Science fiction', 'Horror tales', 'Gothic fi...",https://www.gutenberg.org/ebooks/84,"[{'start': 1, 'end': 40, 'text': 'Frankenstein..."
1,Romeo and Juliet,"Shakespeare, William, 1564-1616","""Romeo and Juliet"" by William Shakespeare is a...","['Vendetta -- Drama', 'Youth -- Drama', 'Veron...",https://www.gutenberg.org/ebooks/1513,"[{'start': 1, 'end': 17, 'text': 'Romeo and Ju..."
2,Pride and Prejudice,"Austen, Jane, 1775-1817","""Pride and Prejudice"" by Jane Austen is a clas...","['England -- Fiction', 'Young women -- Fiction...",https://www.gutenberg.org/ebooks/1342,"[{'start': 1, 'end': 20, 'text': 'Pride and Pr..."
3,"Moby Dick; Or, The Whale","Melville, Herman, 1819-1891","""Moby Dick; Or, The Whale"" by Herman Melville ...","['Whaling -- Fiction', 'Sea stories', 'Psychol...",https://www.gutenberg.org/ebooks/2701,"[{'start': 1, 'end': 25, 'text': 'Moby Dick; O..."
4,Middlemarch,"Eliot, George, 1819-1880","""Middlemarch"" by George Eliot is a novel writt...","['Didactic fiction', 'City and town life -- Fi...",https://www.gutenberg.org/ebooks/145,"[{'start': 1, 'end': 12, 'text': 'Middlemarch'..."


In [14]:
# Extraigo temas posibles de libros para hacer una primera selección según estado de ánimo
libros_subjects_list=[]

for subjects in libros['Subjects']:
  subjects_split = []
  subjects_split = subjects.replace('[', '').replace(']', '').replace("'", '').replace("--",'').split(', ')
  for subject in subjects_split:
    if subject not in libros_subjects_list:
      libros_subjects_list.append(subject)

In [15]:
len(libros_subjects_list)

2176

In [16]:
libros_subjects_list

['Science fiction',
 'Horror tales',
 'Gothic fiction',
 'Scientists  Fiction',
 'Monsters  Fiction',
 'Frankenstein',
 'Victor (Fictitious character)  Fiction',
 '"Frankensteins monster (Fictitious character)  Fiction"',
 'Vendetta  Drama',
 'Youth  Drama',
 'Verona (Italy)  Drama',
 'Juliet (Fictitious character)  Drama',
 'Romeo (Fictitious character)  Drama',
 'Conflict of generations  Drama',
 'Tragedies',
 'England  Fiction',
 'Young women  Fiction',
 'Love stories',
 'Sisters  Fiction',
 'Domestic fiction',
 'Courtship  Fiction',
 'Social classes  Fiction',
 'Whaling  Fiction',
 'Sea stories',
 'Psychological fiction',
 'Ship captains  Fiction',
 'Adventure stories',
 'Mentally ill  Fiction',
 'Ahab',
 'Captain (Fictitious character)  Fiction',
 'Whales  Fiction',
 'Whaling ships  Fiction',
 'Didactic fiction',
 'City and town life  Fiction',
 'Married people  Fiction',
 'Bildungsromans',
 'English drama  Early modern and Elizabethan',
 '1500-1600',
 'Humorous stories',
 'Britis

Según lo explorado en las distintas bases de datos, se define la siguiente estrategia de búsqueda:
* Películas: Según el reconocimiento de emociones, primero se filtrará la base de datos por género.
  - POSITIVO: 'Action', 'Adventure', 'Animation', 'Comedy', 'Romance', 'Musical'
  - NEUTRAL:  'Mystery', 'Sci-Fi', 'Family', 'Fantasy','Music', 'Crime', 'War', 'Sport'
  - NEGATIVO: 'Horror', 'Thriller', 'Drama', 'Biography', 'History', 'Western'

* Juegos: Como se cuenta con muchos géneros, el primer filtro se realizará buscando coincidencias en entidades nombradas. En el caso de que no se encuentre ninguna coincidencia, se utilizará la base de datos completa.

* Libros: La lista de temas también contiene muchas variantes, por lo que se aplicará el mismo criterio que en los juegos.

















### Embeddings generados

In [17]:
# Cargar incrustaciones guardadas
incrustaciones_juegos = np.load(embeddings_juegos_dir)
incrustaciones_peliculas = np.load(embeddings_peliculas_dir)
incrustaciones_libros = np.load(embeddings_libros_dir)

### Clasificación de estado de ánimo

#### Modelo 1: multilingual-uncased-sentiment

Basado en el ejemplo de la teoría de la Unidad 3.

Utilicé un modelo multilingue basado en BERT:
https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

Este modelo clasifica el sentimiento en una escala de 1 a 5. Las podríamos considerar:
* 1: Melancólico
* 3: Neutral
* 5: Alegre


##### Prueba de clasificador

In [18]:
# Cargo el tokenizador y el modelo
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Creo un pipeline de clasificación
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [19]:
# Lista de frases para analizar
frases = [
    "Hoy estoy un poco cansado.",
    "La lluvia me pone de mal humor.",
    "Me gustan los días nublados.",
    "Hoy es un buen día para quedarse en la casa tomando mates.",
    "No me importa qué hagamos hoy.",
    "Las tormentas me asustan."
]

# Obtengo las predicciones de sentimiento para cada frase
for frase in frases:
    result = nlp(frase)
    print(f"Frase: '{frase}'")
    print(f"  Sentimiento: {result[0]['label']}, Score: {result[0]['score']:.3f}")
    print()

Frase: 'Hoy estoy un poco cansado.'
  Sentimiento: 3 stars, Score: 0.596

Frase: 'La lluvia me pone de mal humor.'
  Sentimiento: 1 star, Score: 0.510

Frase: 'Me gustan los días nublados.'
  Sentimiento: 4 stars, Score: 0.540

Frase: 'Hoy es un buen día para quedarse en la casa tomando mates.'
  Sentimiento: 4 stars, Score: 0.426

Frase: 'No me importa qué hagamos hoy.'
  Sentimiento: 2 stars, Score: 0.339

Frase: 'Las tormentas me asustan.'
  Sentimiento: 2 stars, Score: 0.400



#### Modelo 2: pysentimiento

Como alternativa, explorando el repositorio de HugginFace, encontré un modelo entrenado con el dataset "TASS 2020 Task 2" que contiene 8,409 tweets en español etiquetados según las definidas "emociones universales" de Ekman:

* ira
* asco
* miedo
* alegría
* tristeza
* sorpresa

Esta herramienta presenta distintos analizadores:
* Análisis de sentimiento
* Detección de discurso de odio
* Detección de ironía
* Análisis de emoción

Para la consigna se utilizará el *Análisis de sentimiento* que clasifica una frase como Positiva, Negativa o Neutral con un probabilidad para cada caso.

**HuggingFace:** https://huggingface.co/pysentimiento/robertuito-emotion-analysis

**Github:** https://github.com/pysentimiento/pysentimiento/tree/master


##### Prueba de clasificador

In [20]:
analyzer_sentiment = create_analyzer(task="sentiment", lang="es")

config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [21]:
# Análisis de salida
analyzer_sentiment.predict("Qué día espectacular!")

AnalyzerOutput(output=POS, probas={POS: 0.944, NEU: 0.046, NEG: 0.010})

In [22]:
# Lista de frases para analizar
frases = [
    "Hoy estoy un poco cansado.",
    "La lluvia me pone de mal humor.",
    "Me gustan los días nublados.",
    "Hoy es un buen día para quedarse en la casa tomando mates.",
    "No me importa qué hagamos hoy.",
    "Las tormentas me asustan."
]

# Obtengo las predicciones de sentimiento para cada frase
for frase in frases:
    result = analyzer_sentiment.predict(frase)
    print(f"Frase: '{frase}'")
    print(f"  Sentimiento: {result.output}, Score: {result.probas[result.output]:.3f}")
    print()

Frase: 'Hoy estoy un poco cansado.'
  Sentimiento: NEG, Score: 0.938

Frase: 'La lluvia me pone de mal humor.'
  Sentimiento: NEG, Score: 0.982

Frase: 'Me gustan los días nublados.'
  Sentimiento: POS, Score: 0.879

Frase: 'Hoy es un buen día para quedarse en la casa tomando mates.'
  Sentimiento: POS, Score: 0.934

Frase: 'No me importa qué hagamos hoy.'
  Sentimiento: NEU, Score: 0.725

Frase: 'Las tormentas me asustan.'
  Sentimiento: NEG, Score: 0.953



#### Comparación


```
FRASE                                          MODELO 1             MODELO 2
--------------------------------------------------------------------------------
Hoy estoy un poco cansado."                   3 ; 0.596            NEG ; 0.938

"La lluvia me pone de mal humor."             1 ; 0.510            NEG ; 0.982

"Me gustan los días nublados."                4 ; 0.540            POS ; 0.879

"Hoy es un buen día para quedarse en          4 ; 0.426            POS ; 0.934
la casa tomando mates."

"No me importa qué hagamos hoy."              2 ; 0.339            NEU ; 0.725

"Las tormentas me asustan."                   2 ; 0.400            NEG ; 0.953

```

Para el desarrollo del trabajo se utilizará el **Modelo 2: pysentimiento** ya que se adapta mejor a la consigna de clasificar el estado de ánimo en 3 categorías (Positivo, Neutral, Negativo). Por otro lado, comparando los resultados de análisis sobre las mismas frases, considero más acertados los del segundo modelo.

### Idiomas




La interacción con el usuario será en español pero las fuentes de datos se encuentran en inglés.

Lo más conveniente será que el procesamiento se realice en inglés y el resultado final se traduzca a español antes de devolverlo al usuario.

A modo de prueba defino dos opciones de traducción:
1. Modelos NMT EN-ES / ES-EN *(mode=0)*
2. API de Google Translate *(mode=1)*

En las funciones **translate_en_es** y **translate_es_en** se puede ajustar el parámetro *mode* para seleccionar uno u otro.

##### Funciones

In [23]:
# Definimos el modelo y el tokenizador INGLÉS -> ESPAÑOL
modelo_en_es = 'Helsinki-NLP/opus-mt-en-es'
tokenizer_en_es = MarianTokenizer.from_pretrained(modelo_en_es)
model_en_es = MarianMTModel.from_pretrained(modelo_en_es)

# Definimos el modelo y el tokenizador ESPAÑOL -> INGLÉS
modelo_es_en = 'Helsinki-NLP/opus-mt-es-en'
tokenizer_es_en = MarianTokenizer.from_pretrained(modelo_es_en)
model_es_en = MarianMTModel.from_pretrained(modelo_es_en)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [24]:
def translate_en_es(text, mode=1):
  # Modo 0 = Modelo NMT
  if mode == 0:
    # Tokeniza el texto y genera la traducción
    inputs = tokenizer_en_es(text, return_tensors="pt")
    outputs = model_en_es.generate(**inputs)

    # Decodifica y muestra la traducción
    texto_espanol = tokenizer_en_es.decode(outputs[0], skip_special_tokens=True)

  # Modo 1 = Google Translate
  elif mode == 1:
    texto_espanol = GoogleTranslator(source='en', target='es').translate(text)

  return texto_espanol

In [25]:
def translate_es_en(text, mode=1):
  # Modo 0 = Modelo NMT
  if mode == 0:
    # Tokeniza el texto y genera la traducción
    inputs = tokenizer_es_en(text, return_tensors="pt")
    outputs = model_es_en.generate(**inputs)

    # Decodifica y muestra la traducción
    texto_ingles = tokenizer_es_en.decode(outputs[0], skip_special_tokens=True)

  # Modo 1 = Google Translate
  elif mode == 1:
    texto_ingles = GoogleTranslator(source='es', target='en').translate(text)

  return texto_ingles

##### Pruebas

In [26]:
# Prueba Español -> Inglés con modelo NMT
translate_es_en("Quisiera ver alguna película sobre invasiones extraterrestres. Que sea divertida y no dure más de 100 minutos.", mode=0)

"I'd like to see some film about alien invasions that's fun and doesn't last more than 100 minutes."

In [27]:
# Prueba Español -> Inglés con Google Translate
translate_es_en("Quisiera ver alguna película sobre invasiones extraterrestres. Que sea divertida y no dure más de 100 minutos.", mode=1)

"I'd like to see a movie about alien invasions. Something that's fun and doesn't last more than 100 minutes."

In [28]:
# Prueba Inglés -> Español con modelo NMT
translate_en_es("""
Brass: Birmingham is an economic strategy game sequel to Martin Wallace' 2007 masterpiece, Brass.
Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.
It offers a very different story arc and experience from its predecessor.
As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands.
The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs.
VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.
Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):
1) Build - Pay required resources and place an industry tile.
2) Network - Add a rail / canal link, expanding your network.
3) Develop - Increase the VP value of an industry.
4) Sell - Sell your cotton, manufactured goods and pottery.
5) Loan - Take a &pound;30 loan and reduce your income.
Brass: Birmingham also features a new sixth action:
6) Scout - Discard three cards and take a wild location and wild industry card.
(This action replaces Double Action Build in original Brass.)""", mode=0)

'Latón: Birmingham cuenta la historia de los empresarios competidores en Birmingham durante la revolución industrial, entre los años 1770-1870. Ofrece un arco de historia muy diferente y la experiencia de su predecesor. Al igual que en su predecesor, usted debe desarrollar, construir y establecer su industria y red, en un esfuerzo por explotar las demandas bajas o altas del mercado. El juego se juega sobre dos mitades: la era del canal (años 1770-1830) y la era del ferrocarril (años 1830-1870). Para ganar el juego, anotar la mayor parte de los VPs. Los VPs se cuentan al final de cada mitad para los canales, los ferrocarriles y los azulejos de la industria establecidos. Cada ronda, los jugadores se turnan de acuerdo con el orden de turno, recibiendo dos acciones para realizar cualquiera de las siguientes acciones (encontradas en el juego original): 1) Construir - Pagar los recursos necesarios y colocar una baldosa industrial. 2) Red - Añadir un enlace ferrocarril / canal, ampliar su red

In [29]:
# Prueba Inglés -> Español con Google Translate
translate_en_es("""
Brass: Birmingham is an economic strategy game sequel to Martin Wallace' 2007 masterpiece, Brass.
Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.
It offers a very different story arc and experience from its predecessor.
As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands.
The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs.
VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.
Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):
1) Build - Pay required resources and place an industry tile.
2) Network - Add a rail / canal link, expanding your network.
3) Develop - Increase the VP value of an industry.
4) Sell - Sell your cotton, manufactured goods and pottery.
5) Loan - Take a &pound;30 loan and reduce your income.
Brass: Birmingham also features a new sixth action:
6) Scout - Discard three cards and take a wild location and wild industry card.
(This action replaces Double Action Build in original Brass.)""", mode=1)

'Brass: Birmingham es una secuela de la obra maestra de Martin Wallace de 2007, Brass.\nBrass: Birmingham cuenta la historia de empresarios que compiten en Birmingham durante la revolución industrial, entre los años 1770 y 1870.\nOfrece un arco argumental y una experiencia muy diferentes a los de su predecesor.\nAl igual que en su predecesor, debes desarrollar, construir y establecer tus industrias y tu red, en un esfuerzo por explotar las demandas bajas o altas del mercado.\nEl juego se desarrolla en dos mitades: la era de los canales (años 1770-1830) y la era del ferrocarril (años 1830-1870). Para ganar el juego, obtén la mayor cantidad de PV.\nLos PV se cuentan al final de cada mitad para los canales, los ferrocarriles y las fichas de industria establecidas (volteadas).\nEn cada ronda, los jugadores se turnan según el orden de turno y reciben dos acciones para realizar cualquiera de las siguientes acciones (que se encuentran en el juego original):\n1) Construir: paga los recursos ne

##### Comparación

En base a la velocidad y calidad de traducción el modo que devuelve mejores resultados es el **Modo 1: Google Translate.**

### Búsqueda de Opciones

La estrategia de búsqueda elegida es la siguiente:
* Reconocer el estado de ánimo del usuario. El resultado será: *POSITIVO*, *NEUTRAL* o *NEGATIVO*
* Identificar entidades nombradas en las preferencias ingresadas por el usuario.
* Según la emoción identificada, las entidades reconocidas y el tipo de contenido seleccionado, se filtrarán las bases de datos según los géneros/categorías de cada entrada.
* Generar los embeddings de las preferencias ingresadas por el usuario.
* Realizar una búsqueda asimétrica en las descripciones/resúmenes de los elementos de las bases de datos.

In [30]:
# Lista de etiquetas que el modelo intentará encontrar en el texto
labels_pelicula = ["movie","person", "book", "location", "date", "actor", "character", ""]
labels_juego = ["game","mechanic", "character", "piece", "theme", "date", "author", "action", ""]
labels_libro = ["person", "book", "location", "date", "character", ""]

#### Identificación de entidades nombradas - Modelo NER

Utilizo el modelo **gliner-community/gliner_large-v2.5**, el mismo utilizado para complementar las bases de datos.

In [31]:
# Carga de modelo preentrenado
# model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
model = GLiNER.from_pretrained("gliner-community/gliner_large-v2.5")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.87k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

...

models_comparison.png:   0%|          | 0.00/156k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

In [32]:
# Cambia el modelo a modo de evaluación, esto es útil para desactivar características específicas como dropout durante la inferencia
model.eval()

GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(128003, 1024, padding_idx=0)
            (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-23): 24 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
                    (pos_dropout): StableDropout()
                    (dropout): StableDropout()
                  )
                  (output)

#### Búsqueda semántica

Se necesita realizar búsquedas en las bases de datos según la preferencia ingresada por el usuario. Para ello se realizarán **búsquedas asimétricas** en las descripciones/resúmenes de las películas, libros y juegos.

Esto se debe a que se cumplen ambas características mencionadas en la teoría de este tipo de búsquedas:
* Diferencia significativa en la longitud de los textos
* No intercambiabilidad entre la consulta y el corpus


In [33]:
modelo = SentenceTransformer('msmarco-MiniLM-L-6-v3')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Recomendaciones

In [34]:
def foo(sender):
  # Título de la salida
  print("Veamos que puedo encontrar para recomendarte...")
  print('***********************************************************************************************')

  # Lectura de tipo de contenido
  usuario_tipo = widget_tipo.value
  print(f"Contenido seleccionado: {usuario_tipo}")

  # Lectura de cantidad de resultados
  usuario_cantidad = widget_cantidad.value
  print(f"Cantidad seleccionada: {usuario_cantidad}")
  print('***********************************************************************************************')

  # Genero dataframes vacíos que servirán para almacenar contenidos filtrados
  juegos_filtrados = pd.DataFrame(columns=juegos.columns)
  libros_filtrados = pd.DataFrame(columns=libros.columns)
  peliculas_filtradas = pd.DataFrame(columns=peliculas.columns)

  # Lectura de estado de ánimo
  usuario_animo = widget_animo.value
  #print(f"Texto ingresado: {usuario_animo}")
  usuario_animo_result = analyzer_sentiment.predict(usuario_animo)
  #print(f"Sentimiento: {usuario_animo_result.output}, Score: {usuario_animo_result.probas[usuario_animo_result.output]:.3f}")

  # Lectura de preferencias
  usuario_preferencia = widget_preferencia.value
  usuario_preferencia_en = translate_es_en(usuario_preferencia)
  incrustaciones_preferencia=modelo.encode(usuario_preferencia_en)

  # Busco las entidades para compararlos con los libros y juegos
  entities_juego_preferencia = model.predict_entities(usuario_preferencia_en, labels_juego, threshold=0.4)
  entities_libro_preferencia = model.predict_entities(usuario_preferencia_en, labels_libro, threshold=0.4)
  #print(f"Entidades encontradas: {entities_juego_preferencia}, {entities_libro_preferencia}")

  #################################################################################################################################

  #### LIBROS #####################################################################################################################
  if usuario_tipo == 'Libros':
    indices_libros_preferencias = []
    # Busco entidades reconocidas en las preferencias dentro de las entidades de los libros
    for entity in entities_libro_preferencia:
      text_buscado = entity['text']

      # Obtener índices de filas que contengan la entidad buscada
      indices_filtrados_entity = libros[libros['entities'].str.contains(text_buscado, case=False, na=False)].index
      # Agrego índices filtrados a lista general
      indices_libros_preferencias = list(set(indices_libros_preferencias) | set(indices_filtrados_entity))

    # Selecciono sólo las incrustaciones de los libros que coinciden con alguna entidad reconocida
    if len(indices_libros_preferencias) > 0:
      # Genero un dataframe filtrado y reseteo los índices para que coincida con las incrustaciones filtradas
      libros_filtrados = libros.loc[indices_libros_preferencias].reset_index()
      incrustaciones_libros_filtradas = [incrustaciones_libros[i] for i in indices_libros_preferencias]
    else:
      incrustaciones_libros_filtradas = incrustaciones_libros
      libros_filtrados = libros

    # Búsqueda semántica en incrustaciones
    hits_libros = util.semantic_search(incrustaciones_preferencia, incrustaciones_libros_filtradas, top_k=int(usuario_cantidad))

    for j, hit in enumerate(hits_libros[0]):
      titulo = translate_en_es(libros_filtrados.loc[hit['corpus_id'],'Title'], mode=1)
      titulo_original = libros_filtrados.loc[hit['corpus_id'],'Title']
      autor = libros_filtrados.loc[hit['corpus_id'],'Author']
      resumen = translate_en_es(libros_filtrados.loc[hit['corpus_id'],'Summary'], mode=1)
      temas = translate_en_es(libros_filtrados.loc[hit['corpus_id'],'Subjects'], mode=1)
      url = libros_filtrados.loc[hit['corpus_id'],'URL']

      print(f"Respuesta {j+1} (Similitud: {hit['score']:.4f})")
      print(f"Título: {titulo}")
      print(f"Título original: {titulo_original}")
      print(f"Autor: {autor}")
      print(f"Resumen: {resumen}")
      print(f"Temas: {temas}")
      print(f"URL: {url}")
      print('-----------------------------------------------------------------------------------------------')
    print()

  #### JUEGOS #####################################################################################################################
  elif usuario_tipo == 'Juegos':
    indices_juegos_preferencias = []
    # Busco entidades reconocidas en las preferencias dentro de las entidades de los juegos
    for entity in entities_juego_preferencia:
      text_buscado = entity['text']

      # Obtener índices de filas que contengan la entidad buscada
      indices_filtrados_entity = juegos[juegos['entities'].str.contains(text_buscado, case=False, na=False)].index
      # Agrego índices filtrados a lista general
      indices_juegos_preferencias = list(set(indices_juegos_preferencias) | set(indices_filtrados_entity))

    # Selecciono sólo las incrustaciones de los juegos que coinciden con alguna entidad reconocida
    if len(indices_juegos_preferencias) > 0:
      # Genero un dataframe filtrado y reseteo los índices para que coincida con las incrustaciones filtradas
      juegos_filtrados = juegos.loc[indices_juegos_preferencias].reset_index()
      incrustaciones_juegos_filtradas = [torch.from_numpy(incrustaciones_juegos[i]) for i in indices_juegos_preferencias]
    else:
      incrustaciones_juegos_filtradas = [torch.from_numpy(emb) for emb in incrustaciones_juegos]
      juegos_filtrados = juegos

    # Búsqueda semántica en incrustaciones
    hits_juegos = util.semantic_search(incrustaciones_preferencia, incrustaciones_juegos_filtradas, top_k=int(usuario_cantidad))

    # Muestro los resultados en pantalla ordenados y traducidos
    for j, hit in enumerate(hits_juegos[0]):
      nombre = translate_en_es(juegos_filtrados.loc[hit['corpus_id'],'game_name'], mode=1)
      nombre_original = juegos_filtrados.loc[hit['corpus_id'],'game_name']
      avg_rating = juegos_filtrados.loc[hit['corpus_id'],'avg_rating']
      descripcion = translate_en_es(juegos_filtrados.loc[hit['corpus_id'],'description'], mode=1)
      desarrollador = juegos_filtrados.loc[hit['corpus_id'],'designers']
      anio = juegos_filtrados.loc[hit['corpus_id'],'yearpublished']
      categorias = translate_en_es(juegos_filtrados.loc[hit['corpus_id'],'categories'])
      mecanicas = translate_en_es(juegos_filtrados.loc[hit['corpus_id'],'mechanics'])
      tiempo_min = juegos_filtrados.loc[hit['corpus_id'],'minplaytime']
      tiempo_max = juegos_filtrados.loc[hit['corpus_id'],'maxplaytime']
      num_jugadores_min = juegos_filtrados.loc[hit['corpus_id'],'minplayers']
      num_jugadores_max = juegos_filtrados.loc[hit['corpus_id'],'maxplayers']

      print(f"Respuesta {j+1} (Similitud: {hit['score']:.4f})")
      print(f"Nombre: {nombre}")
      print(f"Nombre original: {nombre_original}")
      print(f"Rating promedio: {avg_rating}")
      print(f"Descripción: {descripcion}")
      print(f"Desarrollador: {desarrollador}")
      print(f"Año: {anio}")
      print(f"Categorías: {categorias}")
      print(f"Mecánicas: {mecanicas}")
      print(f"Tiempo[min]: {tiempo_min} a {tiempo_max}")
      print(f"Número de jugadores: {num_jugadores_min} a {num_jugadores_max}")
      print('-----------------------------------------------------------------------------------------------')
    print()



  #### PELICULAS ####################################################################################################################
  elif usuario_tipo == 'Películas':
    indices_películas_animo_usuario = []
    # Si el estado de ánimo es POSITIVO
    if usuario_animo_result.output == "POS":
      print("Veo que estas con una actitud positiva!")
      print('-----------------------------------------------------------------------------------------------')
      # Busco películas de los géneros 'Action', 'Adventure', 'Animation', 'Comedy', 'Romance', 'Musical'
      indices_películas_animo_usuario = peliculas[peliculas['Genre'].str.contains('Action|Adventure|Animation|Comedy|Romance|Musical', case=False, na=False)].index

    # Si el estado de ánimo es NEUTRAL
    elif usuario_animo_result.output == "NEU":
      print("Tu ánimo es bastante equilibrado.")
      print('-----------------------------------------------------------------------------------------------')
      # Busco películas de los géneros 'Mystery', 'Sci-Fi', 'Family', 'Fantasy','Music', 'Crime', 'War', 'Sport'
      indices_películas_animo_usuario = peliculas[peliculas['Genre'].str.contains('Mystery|Sci-Fi|Family|Fantasy|Music|Crime|War|Sport', case=False, na=False)].index

    # Si el estado de ánimo es NEGATIVO
    elif usuario_animo_result.output == "NEG":
      print("Te noto un poco desanimado...")
      print('-----------------------------------------------------------------------------------------------')
      # Busco películas de los géneros 'Horror', 'Thriller', 'Drama', 'Biography', 'History', 'Western'
      indices_películas_animo_usuario = peliculas[peliculas['Genre'].str.contains('Horror|Thriller|Drama|Biography|History|Western', case=False, na=False)].index

    indices_películas_animo_usuario = list(indices_películas_animo_usuario)

    # Selecciono sólo las incrustaciones de los juegos que coinciden con alguna entidad reconocida
    if len(indices_películas_animo_usuario) > 0:
      # Genero un dataframe filtrado y reseteo los índices para que coincida con las incrustaciones filtradas
      peliculas_filtradas = peliculas.loc[indices_películas_animo_usuario].reset_index()
      incrustaciones_peliculas_filtradas = [torch.from_numpy(incrustaciones_peliculas[i]) for i in indices_películas_animo_usuario]
    else:
      incrustaciones_juegos_filtradas = [torch.from_numpy(emb) for emb in incrustaciones_peliculas]
      peliculas_filtradas = peliculas


    # Búsqueda semántica en incrustaciones
    hits_peliculas = util.semantic_search(incrustaciones_preferencia, incrustaciones_peliculas_filtradas, top_k=int(usuario_cantidad))

    # Muestro los resultados en pantalla ordenados y traducidos
    for j, hit in enumerate(hits_peliculas[0]):
      titulo = translate_en_es(peliculas_filtradas.loc[hit['corpus_id'],'Title'], mode=1)
      titulo_original = peliculas_filtradas.loc[hit['corpus_id'],'Title']
      genero = translate_en_es(peliculas_filtradas.loc[hit['corpus_id'],'Genre'], mode=1)
      descripcion = translate_en_es(peliculas_filtradas.loc[hit['corpus_id'],'Description'], mode=1)
      director = peliculas_filtradas.loc[hit['corpus_id'],'Director']
      actores = peliculas_filtradas.loc[hit['corpus_id'],'Actors']
      anio = peliculas_filtradas.loc[hit['corpus_id'],'Year']
      duracion = peliculas_filtradas.loc[hit['corpus_id'],'Runtime (Minutes)']
      rating = peliculas_filtradas.loc[hit['corpus_id'],'Rating']

      print(f"Respuesta {j+1} (Similitud: {hit['score']:.4f})")
      print(f"Título: {titulo}")
      print(f"Título original: {titulo_original}")
      print(f"Género: {genero}")
      print(f"Descripción: {descripcion}")
      print(f"Director: {director}")
      print(f"Actores: {actores}")
      print(f"Año: {anio}")
      print(f"Duración: {duracion}")
      print(f"Rating: {rating}")
      print('-----------------------------------------------------------------------------------------------')
    print()

### Interfaz de usuario

In [36]:
########################################################################################################
# ESTADO DE ÁNIMO
########################################################################################################
# Etiqueta del widget
label_animo = widgets.Label(
    value="¿Qué estás pensando?")

# Widget de entrada de texto
widget_animo = widgets.Text(
    value='',                                 # Valor inicial
    placeholder='Respuesta...',               # Texto a mostrar cuando el widget está vacío
    disabled=False,
    layout=widgets.Layout(width='500px')      # Ajustar el ancho del widget
)

# Mostrar el widget
display(label_animo,widget_animo)

########################################################################################################
# PREFERENCIAS
########################################################################################################
# Etiqueta del widget
label_preferencia = widgets.Label(
    value="¿Cuáles son tus preferencias?")

# Widget de entrada de texto
widget_preferencia = widgets.Text(
    value='',                                 # Valor inicial
    placeholder='Respuesta...',               # Texto a mostrar cuando el widget está vacío
    disabled=False,
    layout=widgets.Layout(width='500px')      # Ajustar el ancho del widget
)

# Mostrar el widget
display(label_preferencia,widget_preferencia)

########################################################################################################
# SELECCIÓN DE RECOMENDACIÓN
########################################################################################################
# Etiqueta del widget
label_tipo = widgets.Label(
    value="¿Qué tipo de contenido quieres que te recomiende?")

# Widget de selección de contenido
widget_tipo = widgets.Select(
    options=['Libros', 'Películas', 'Juegos'],
    value='Libros',
    disabled=False,
    layout=widgets.Layout(width='500px')      # Ajustar el ancho del widget
)

# Mostrar el widget
display(label_tipo,widget_tipo)

########################################################################################################
# SELECCIÓN DE CANTIDAD
########################################################################################################
# Etiqueta del widget
label_cantidad = widgets.Label(
    value="¿Cuántas opciones quieres que busque?")

# Widget de selección de cantidad
widget_cantidad = widgets.Dropdown(
    options=['1', '2', '3', '4', '5'],
    value='1',
    disabled=False,
    layout=widgets.Layout(width='500px')      # Ajustar el ancho del widget
)

# Mostrar el widget
display(label_cantidad,widget_cantidad)

########################################################################################################
# CONFIRMAR DATOS
########################################################################################################
# Etiqueta del widget
label_boton = widgets.Label(
    value="")

button = widgets.Button(
    description='Buscar recomendaciones',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(width='500px')      # Ajustar el ancho del widget
)
display(label_boton,button)

button.on_click(foo)




Label(value='¿Qué estás pensando?')

Text(value='', layout=Layout(width='500px'), placeholder='Respuesta...')

Label(value='¿Cuáles son tus preferencias?')

Text(value='', layout=Layout(width='500px'), placeholder='Respuesta...')

Label(value='¿Qué tipo de contenido quieres que te recomiende?')

...

Label(value='¿Cuántas opciones quieres que busque?')

Dropdown(layout=Layout(width='500px'), options=('1', '2', '3', '4', '5'), value='1')

Label(value='')

Button(button_style='info', description='Buscar recomendaciones', icon='check', layout=Layout(width='500px'), …

Veamos que puedo encontrar para recomendarte...
***********************************************************************************************
Contenido seleccionado: Juegos
Cantidad seleccionada: 4
***********************************************************************************************
Respuesta 1 (Similitud: 0.4898)
Nombre: Proyecto L
Nombre original: Project L
Rating promedio: 7.49
Descripción: ¡Construye piezas, desarrolla un motor, perfecciona tu estrategia y gana el juego!

Project L es un juego de combinación de piezas de ritmo rápido con rompecabezas 3D de tres capas y hermosas piezas de acrílico. Desafía a tus amigos a un juego de diseño simple pero con una jugabilidad intrincada que deja una impresión duradera.

La base del juego radica en usar tus piezas para completar rompecabezas. Comenzando con solo dos piezas básicas, usas tres acciones en cada turno para desarrollar un motor poderoso. Con más piezas de varios tipos, puedes completar de manera eficiente incluso l